In [41]:
import pandas as pd
import os
import time

## Get sector data from Kaggle
https://www.kaggle.com/dgawlik/nyse#securities.csv


In [3]:
sec = pd.read_csv('./securities.csv')
sec.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


### We only need the unique tickers and the GICS Sector

Ref: https://www.fidelity.com/learning-center/trading-investing/markets-sectors/global-industry-classification-standard


In [30]:
sec = sec[['Ticker symbol', 'GICS Sector']]
sec = sec.drop_duplicates()
sec.head()
# list(sec['Ticker symbol'])
pd.unique(sec['GICS Sector'])

array(['Industrials', 'Health Care', 'Information Technology',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Consumer Staples', 'Real Estate', 'Energy',
       'Telecommunications Services'], dtype=object)

### We need to make directories for each sector

In [31]:
sec.head()

,Ticker symbol,GICS Sector
0,MMM,Industrials
1,ABT,Health Care
2,ABBV,Health Care
3,ACN,Information Technology
4,ATVI,Information Technology


In [38]:
# path_to_file = './csv/2017-01-03/'
h5file_path = './data/book_events_total_view_2017-01-03.h5'

csv_file_name = h5file_path.split('/')[2].split('.')[0]

### We store one day's worth of data into on csv file with the appropriate ticker and sector given for each traded stock

In [42]:
start_time = time.time()
csv_path_to_file = './csv/2017-01-03/'
with pd.HDFStore(h5file_path, 'r') as train:
    final_df = pd.DataFrame()
    for key in train.keys():
#         path_to_file = './csv/2017-01-03/'
        key2 = key[1:]
        if key2 in list(sec['Ticker symbol']):
            df = train.get(key)
            df = df[df['book_event_type']=='T']
            # get sector
            sector = sec[sec['Ticker symbol'] == key2]['GICS Sector'].iloc[0]
            df['sector'] = sector
            df['ticker'] = key2
#             path_to_file += sector
#             if not os.path.exists(path_to_file):
#                 os.mkdir(path_to_file)
            final_df = final_df.append(df)
            
    final_df.to_csv(csv_path_to_file + '/' + csv_file_name +'.csv')

print("--- %s seconds ---" % (time.time() - start_time))

--- 552.9635646343231 seconds ---
